In [1]:
# from dash import Dash, html, dcc, dash_table, callback, Output, Input, State, ctx
# import pandas as pd
# import plotly.express as px

# # Create Dash app
# app = Dash(__name__)

# # Layout
# app.layout = html.Div([
#     # Chart type selector
#     dcc.Dropdown(
#         id="chart-type",
#         options=[
#             {"label": "Line", "value": "line"},
#             {"label": "Scatter", "value": "scatter"},
#             {"label": "Histogram", "value": "hist"},
#             {"label": "Violin", "value": "violin"}
#         ],
#         value="line",
#         clearable=False
#     ),

#     # Variable selector
#     dcc.Dropdown(
#         id="var-select",
#         options=["X", "Y", "Z", "all"],
#         value=["X"],
#         multi=True
#     ),

#     # Sample size input
#     dcc.Input(id="sample-size", type="number", min=10, step=10, value=200),

#     # Navigation buttons
#     html.Button("Prev", id="btn-prev"),
#     html.Button("Next", id="btn-next"),

#     # Pager store
#     dcc.Store(id="pager", data={"start": 0}),

#     # Graph
#     dcc.Graph(id="main-graph"),

#     # Data table
#     dash_table.DataTable(id="data-table")
# ])

# # Single callback for graph, table, and pager
# @callback(
#     Output("main-graph", "figure"),
#     Output("data-table", "data"),
#     Output("pager", "data"),
#     Input("chart-type", "value"),
#     Input("var-select", "value"),
#     Input("sample-size", "value"),
#     Input("btn-prev", "n_clicks"),
#     Input("btn-next", "n_clicks"),
#     State("pager", "data")
# )
# def update_view(ctype, vars_sel, N, n_prev, n_next, pager):
#     # Load and clean data
#     df = pd.read_csv('gyroscope_data_cleaned.csv')
#     df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce", utc=True)
#     for col in ["X", "Y", "Z"]:
#         df[col] = pd.to_numeric(df[col], errors="coerce")
#     df = df.dropna(subset=["Timestamp"]).sort_values("Timestamp")
#     len_df = len(df)

#     # Pagination logic
#     start = pager.get("start", 0) if pager else 0
#     max_start = max(0, len_df - N)

#     if ctx.triggered_id == "btn-next":
#         start = min(start + N, max_start)
#     elif ctx.triggered_id == "btn-prev":
#         start = max(start - N, 0)

#     # Current view
#     view = df.iloc[start:start + N]

#     # Variable selection
#     if vars_sel == "all" or (isinstance(vars_sel, list) and "all" in vars_sel):
#         y_vars = ["X", "Y", "Z"]
#     elif isinstance(vars_sel, list):
#         y_vars = vars_sel
#     else:
#         y_vars = [vars_sel]

#     # Build figure
#     if ctype == "line":
#         fig = px.line(view, x="Timestamp", y=y_vars)

#     elif ctype == "scatter":
#         # compare two axes; if only one selected, use the same for x & y (diagonal)
#         x_axis = y_vars[0]
#         y_axis = y_vars[1] if len(y_vars) > 1 else y_vars[0]
#         fig = px.scatter(view, x=x_axis, y=y_axis)

#     elif ctype == "hist":
#         # long-form so each axis gets its own colour
#         long = view.melt(
#             id_vars="Timestamp",
#             value_vars=y_vars,
#             var_name="Axis",
#             value_name="Value"
#         )
#         fig = px.histogram(long, x="Value", color="Axis", nbins=60)
#         # overlay bars so overlapping colours are visible
#         fig.update_layout(barmode="overlay")

#     elif ctype == "violin":
#         # long-form so each axis gets its own colour
#         long = view.melt(
#             id_vars="Timestamp",
#             value_vars=y_vars,
#             var_name="Axis",
#             value_name="Value"
#         )
#         fig = px.violin(
#             long, x="Axis", y="Value", color="Axis",
#             box=True, points="suspectedoutliers"
#         )

#     else:
#         fig = px.line()

#     # Prepare table data (last 10 rows of current window)
#     table_data = view.tail(10).to_dict("records")

#     return fig, table_data, {"start": start}

# if __name__ == "__main__":
#     app.run(debug=True)


In [2]:
from dash import Dash, html, dcc, dash_table, callback, Output, Input, State, ctx
import pandas as pd
import plotly.express as px

# Create Dash app
app = Dash(__name__)

# Layout
app.layout = html.Div([
    dcc.Dropdown(
        id="chart-type",
        options=[
            {"label": "Line", "value": "line"},
            {"label": "Scatter", "value": "scatter"},
            {"label": "Histogram", "value": "hist"},
            {"label": "Violin", "value": "violin"}
        ],
        value="line",
        clearable=False
    ),
    dcc.Dropdown(
        id="var-select",
        options=["X", "Y", "Z", "all"],
        value=["X"],
        multi=True
    ),
    dcc.Input(id="sample-size", type="number", min=10, step=10, value=200),
    html.Button("Prev", id="btn-prev"),
    html.Button("Next", id="btn-next"),
    dcc.Store(id="pager", data={"start": 0}),

    dcc.Graph(id="main-graph"),

    html.Hr(),
    html.H4("Summary (current view)"),
    dash_table.DataTable(id="summary-table", page_size=6, style_table={"maxWidth": 700}),
    html.Hr(),
    html.H4("Last 10 rows (current view)"),
    dash_table.DataTable(id="data-table", page_size=10)
])

# Single callback for graph, tables, and pager
@callback(
    Output("main-graph", "figure"),
    Output("summary-table", "data"),
    Output("summary-table", "columns"),
    Output("data-table", "data"),
    Output("pager", "data"),
    Input("chart-type", "value"),
    Input("var-select", "value"),
    Input("sample-size", "value"),
    Input("btn-prev", "n_clicks"),
    Input("btn-next", "n_clicks"),
    State("pager", "data")
)
def update_view(ctype, vars_sel, N, n_prev, n_next, pager):
    # Load + clean
    df = pd.read_csv('gyroscope_data_cleaned.csv')
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce", utc=True)
    for col in ["X", "Y", "Z"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    df = df.dropna(subset=["Timestamp"]).sort_values("Timestamp")
    len_df = len(df)

    # Paging
    start = pager.get("start", 0) if pager else 0
    max_start = max(0, len_df - N)
    if ctx.triggered_id == "btn-next":
        start = min(start + N, max_start)
    elif ctx.triggered_id == "btn-prev":
        start = max(start - N, 0)

    view = df.iloc[start:start + N]

    # Variable selection → y_vars list
    if vars_sel == "all" or (isinstance(vars_sel, list) and "all" in vars_sel):
        y_vars = ["X", "Y", "Z"]
    elif isinstance(vars_sel, list):
        y_vars = vars_sel
    else:
        y_vars = [vars_sel]

    # -------- Figure --------
    if ctype == "line":
        fig = px.line(view, x="Timestamp", y=y_vars)

    elif ctype == "scatter":
        x_axis = y_vars[0]
        y_axis = y_vars[1] if len(y_vars) > 1 else y_vars[0]
        fig = px.scatter(view, x=x_axis, y=y_axis)

    elif ctype == "hist":
        long = view.melt(id_vars="Timestamp", value_vars=y_vars,
                         var_name="Axis", value_name="Value")
        fig = px.histogram(long, x="Value", color="Axis", nbins=60)
        fig.update_layout(barmode="overlay")

    elif ctype == "violin":
        long = view.melt(id_vars="Timestamp", value_vars=y_vars,
                         var_name="Axis", value_name="Value")
        fig = px.violin(long, x="Axis", y="Value", color="Axis",
                        box=True, points="suspectedoutliers")
    else:
        fig = px.line()

    # -------- Summary table (live) --------
    def to_dash_table(df_sum):
        df_sum = df_sum.reset_index().rename(columns={"index": "variable"})
        df_sum = df_sum.round(3)
        cols = [{"name": c, "id": c} for c in df_sum.columns]
        return df_sum.to_dict("records"), cols

    metrics = ["count", "mean", "std", "min", "median", "max"]

    if ctype in ("line",):
        # summarize selected axes over current window
        df_sum = view[y_vars].agg(metrics).T
        summary_data, summary_cols = to_dash_table(df_sum)

    elif ctype == "scatter":
        x_axis = y_vars[0]
        y_axis = y_vars[1] if len(y_vars) > 1 else y_vars[0]
        df_sum = view[[x_axis, y_axis]].agg(metrics).T
        summary_data, summary_cols = to_dash_table(df_sum)

    elif ctype in ("hist", "violin"):
        long = view.melt(id_vars="Timestamp", value_vars=y_vars,
                         var_name="Axis", value_name="Value")
        df_sum = (long.groupby("Axis")["Value"]
                        .agg(metrics))
        summary_data, summary_cols = to_dash_table(df_sum)

    else:
        df_sum = view[y_vars].agg(metrics).T
        summary_data, summary_cols = to_dash_table(df_sum)

    # -------- Raw rows table (last 10 of current view) --------
    table_data = view.tail(10).to_dict("records")

    return fig, summary_data, summary_cols, table_data, {"start": start}

if __name__ == "__main__":
    app.run(debug=True)


In [3]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.
  Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl.metadata (592 bytes)
Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl (434 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.74.0 requires protobuf<7.0.0,>=6.31.1, but you have protobuf 5.29.5 which is incompatible.


In [4]:
pip install --upgrade protobuf


  Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl (435 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.31.1 which is incompatible.


In [5]:
pip install streamlit protobuf<6 --force-reinstall

Note: you may need to restart the kernel to use updated packages.


The system cannot find the file specified.


SyntaxError: invalid syntax (1389240789.py, line 1)